# Import and Set Up

In [0]:
#Authenticate into Google Drive and find file ids in order to read in data files
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [7]:
import pandas as pd 
import numpy as np 
import statsmodels.api as sm  
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from math import sqrt 
import tensorflow as tf
import time
from sklearn.metrics import accuracy_score



get_ipython().magic(u'matplotlib inline')
#these are some options to control output on your screen 
pd.set_option('display.notebook_repr_html', False) 
pd.set_option('display.max_columns', 40) 
pd.set_option('display.max_rows', 50) 
pd.set_option('display.width', 120)

/usr/local/lib/python2.7/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [0]:
# seed value for random number generators to obtain reproducible results
RANDOM_SEED = 1

In [0]:
# function to clear output console
def clear():
    print("\033[H\033[J")

In [4]:
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

mnist = read_data_sets('MNIST_data')
mnist_X = np.concatenate([mnist.train.images, mnist.validation.images, mnist.test.images])
mnist_y = np.concatenate([mnist.train.labels, mnist.validation.labels, mnist.test.labels])

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
mnist_X.dtype

dtype('float32')

In [6]:
mnist_y.dtype

dtype('uint8')

In [0]:
mnist_y=mnist_y.astype(int)

# EDA

In [8]:
print('\n Structure of explanatory variable array:', mnist_X.shape)
print('\n Structure of response array:', mnist_y.shape)

('\n Structure of explanatory variable array:', (70000, 784))
('\n Structure of response array:', (70000,))


In [0]:
X_train, X_test, y_train, y_test = mnist_X[:60000], mnist_X[60000:], mnist_y[:60000], mnist_y[60000:]

In [0]:
shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

# Analysis

In [0]:
#fitting a neural network to the MNIST data

In [12]:
config = tf.contrib.learn.RunConfig(tf_random_seed=42) # not shown in the config
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please convert numpy dtypes explicitly.
Instructions for updating:
Please specify feature columns explicitly.


In [0]:
##assess classification performance accuracy and processing time.

start = time.time()
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300,100], n_classes=10,
                                         feature_columns=feature_cols, config=config)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf) # if TensorFlow >= 1.1
dnn_clf.fit(X_train, y_train, batch_size=50, steps=40000)
end = time.time()
end-start

In [14]:
#training accuracy
dnn_clf.score(X_train,y_train)

INFO:tensorflow:Starting evaluation at 2018-08-05-14:33:27
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4LLzmK/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-05-14:33:29
INFO:tensorflow:Saving dict for global step 40000: accuracy = 1.0, global_step = 40000, loss = 0.00048598158


{'accuracy': 1.0, 'global_step': 40000, 'loss': 0.00048598158}

In [15]:
yt_pred = dnn_clf.predict(X_train)
accuracy_score(y_train, yt_pred['classes'])

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4LLzmK/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


1.0

In [16]:
#test accuracy
dnn_clf.score(X_test,y_test)

INFO:tensorflow:Starting evaluation at 2018-08-05-14:34:51
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4LLzmK/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-05-14:34:52
INFO:tensorflow:Saving dict for global step 40000: accuracy = 0.9832, global_step = 40000, loss = 0.0683699


{'accuracy': 0.9832, 'global_step': 40000, 'loss': 0.0683699}

In [17]:
y_pred = dnn_clf.predict(X_test)
accuracy_score(y_test, y_pred['classes'])

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4LLzmK/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


0.9832

In [0]:
#testing alternative network structures, activation functions, optimization methods, and/or hyperparameter settings

In [0]:
##assess classification performance accuracy and processing time.

start = time.time()
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300,300], n_classes=10,
                                         feature_columns=feature_cols, config=config)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf) # if TensorFlow >= 1.1
dnn_clf.fit(X_train, y_train, batch_size=50, steps=40000)
end = time.time()
end-start

In [20]:
#training accuracy
dnn_clf.score(X_train,y_train)

INFO:tensorflow:Starting evaluation at 2018-08-05-14:49:20
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpE0Q0o3/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-05-14:49:22
INFO:tensorflow:Saving dict for global step 40000: accuracy = 1.0, global_step = 40000, loss = 0.00038670935


{'accuracy': 1.0, 'global_step': 40000, 'loss': 0.00038670935}

In [21]:
yt_pred = dnn_clf.predict(X_train)
accuracy_score(y_train, yt_pred['classes'])

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpE0Q0o3/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


1.0

In [22]:
#test accuracy
dnn_clf.score(X_test,y_test)

INFO:tensorflow:Starting evaluation at 2018-08-05-14:49:25
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpE0Q0o3/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-05-14:49:26
INFO:tensorflow:Saving dict for global step 40000: accuracy = 0.9829, global_step = 40000, loss = 0.07188314


{'accuracy': 0.9829, 'global_step': 40000, 'loss': 0.07188314}

In [23]:
y_pred = dnn_clf.predict(X_test)
accuracy_score(y_test, y_pred['classes'])

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpE0Q0o3/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


0.9829

# layers with 100 nodes each

In [0]:
##assess classification performance accuracy and processing time.

start = time.time()
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[100,100,100], n_classes=10,
                                         feature_columns=feature_cols, config=config)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf) # if TensorFlow >= 1.1
dnn_clf.fit(X_train, y_train, batch_size=50, steps=40000)
end = time.time()
end-start

In [25]:
#training accuracy
dnn_clf.score(X_train,y_train)

INFO:tensorflow:Starting evaluation at 2018-08-05-14:52:12
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpxqoc2C/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-05-14:52:13
INFO:tensorflow:Saving dict for global step 40000: accuracy = 1.0, global_step = 40000, loss = 0.0002791477


{'accuracy': 1.0, 'global_step': 40000, 'loss': 0.0002791477}

In [26]:
yt_pred = dnn_clf.predict(X_train)
accuracy_score(y_train, yt_pred['classes'])

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpxqoc2C/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


1.0

In [27]:
#test accuracy
dnn_clf.score(X_test,y_test)

INFO:tensorflow:Starting evaluation at 2018-08-05-14:52:16
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpxqoc2C/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-05-14:52:16
INFO:tensorflow:Saving dict for global step 40000: accuracy = 0.9815, global_step = 40000, loss = 0.1004074


{'accuracy': 0.9815, 'global_step': 40000, 'loss': 0.1004074}

In [28]:
y_pred = dnn_clf.predict(X_test)
accuracy_score(y_test, y_pred['classes'])

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpxqoc2C/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


0.9815

4 layers with 500,300,100,50 nodes

In [0]:
##assess classification performance accuracy and processing time.

start = time.time()
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[500,300,100,50], n_classes=10,
                                         feature_columns=feature_cols, config=config)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf) # if TensorFlow >= 1.1
dnn_clf.fit(X_train, y_train, batch_size=50, steps=40000)
end = time.time()
end-start

In [30]:
#training accuracy
dnn_clf.score(X_train,y_train)

INFO:tensorflow:Starting evaluation at 2018-08-05-14:59:06
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9_gzGa/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-05-14:59:08
INFO:tensorflow:Saving dict for global step 40000: accuracy = 1.0, global_step = 40000, loss = 4.4731372e-05


{'accuracy': 1.0, 'global_step': 40000, 'loss': 4.4731372e-05}

In [31]:
yt_pred = dnn_clf.predict(X_train)
accuracy_score(y_train, yt_pred['classes'])

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9_gzGa/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


1.0

In [32]:
#test accuracy
dnn_clf.score(X_test,y_test)

INFO:tensorflow:Starting evaluation at 2018-08-05-14:59:13
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9_gzGa/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-05-14:59:13
INFO:tensorflow:Saving dict for global step 40000: accuracy = 0.9846, global_step = 40000, loss = 0.08410017


{'accuracy': 0.9846, 'global_step': 40000, 'loss': 0.08410017}

In [33]:
y_pred = dnn_clf.predict(X_test)
accuracy_score(y_test, y_pred['classes'])

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9_gzGa/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


0.9846

In [0]:
#try all of this with scaled data

In [0]:
#try a different scaling method to see if that works
#MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

In [0]:
#Lets standardize prior to PCA and PCA on training set
mscaler=MinMaxScaler()
#X_scale=scaler.fit_transform(X)
mscaler.fit(X_train)
X_train_mscaled=mscaler.transform(X_train)
X_test_mscaled=mscaler.transform(X_test)

In [37]:
##assess classification performance accuracy and processing time.

start = time.time()
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300,100], n_classes=10,
                                         feature_columns=feature_cols, config=config)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf) # if TensorFlow >= 1.1
dnn_clf.fit(X_train_mscaled, y_train, batch_size=50, steps=40000)
end = time.time()
end-start

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f435dc74610>, '_model_dir': '/tmp/tmpJfpoIn', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': 42, '_save_summary_steps': 100, '_device_fn': None, '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_environment': 'local', '_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpJfpoIn/model.ckpt.
INFO:tensorflow:loss = 2.311769, step = 1
INFO:tensorflow:global_step/sec: 205.

INFO:tensorflow:loss = 0.062025677, step = 2201 (0.405 sec)
INFO:tensorflow:global_step/sec: 238.422
INFO:tensorflow:loss = 0.0817397, step = 2301 (0.419 sec)
INFO:tensorflow:global_step/sec: 237.417
INFO:tensorflow:loss = 0.042997524, step = 2401 (0.421 sec)
INFO:tensorflow:global_step/sec: 238.023
INFO:tensorflow:loss = 0.0386536, step = 2501 (0.421 sec)
INFO:tensorflow:global_step/sec: 218.524
INFO:tensorflow:loss = 0.11867508, step = 2601 (0.458 sec)
INFO:tensorflow:global_step/sec: 235.517
INFO:tensorflow:loss = 0.0832029, step = 2701 (0.424 sec)
INFO:tensorflow:global_step/sec: 240.705
INFO:tensorflow:loss = 0.04444643, step = 2801 (0.414 sec)
INFO:tensorflow:global_step/sec: 243.482
INFO:tensorflow:loss = 0.032744445, step = 2901 (0.413 sec)
INFO:tensorflow:global_step/sec: 247.903
INFO:tensorflow:loss = 0.04564888, step = 3001 (0.400 sec)
INFO:tensorflow:global_step/sec: 252.255
INFO:tensorflow:loss = 0.06318544, step = 3101 (0.397 sec)
INFO:tensorflow:global_step/sec: 250.684


INFO:tensorflow:loss = 0.020817934, step = 5301 (0.409 sec)
INFO:tensorflow:global_step/sec: 255.021
INFO:tensorflow:loss = 0.11988892, step = 5401 (0.392 sec)
INFO:tensorflow:global_step/sec: 260.243
INFO:tensorflow:loss = 0.02064723, step = 5501 (0.385 sec)
INFO:tensorflow:global_step/sec: 295.852
INFO:tensorflow:loss = 0.004703219, step = 5601 (0.340 sec)
INFO:tensorflow:global_step/sec: 294.348
INFO:tensorflow:loss = 0.011123795, step = 5701 (0.337 sec)
INFO:tensorflow:global_step/sec: 313.478
INFO:tensorflow:loss = 0.09007611, step = 5801 (0.322 sec)
INFO:tensorflow:global_step/sec: 310.413
INFO:tensorflow:loss = 0.07978357, step = 5901 (0.320 sec)
INFO:tensorflow:global_step/sec: 293.321
INFO:tensorflow:loss = 0.030832144, step = 6001 (0.344 sec)
INFO:tensorflow:global_step/sec: 296.573
INFO:tensorflow:loss = 0.033288397, step = 6101 (0.338 sec)
INFO:tensorflow:global_step/sec: 289.267
INFO:tensorflow:loss = 0.015925495, step = 6201 (0.343 sec)
INFO:tensorflow:global_step/sec: 27

INFO:tensorflow:loss = 0.0030981686, step = 8401 (0.404 sec)
INFO:tensorflow:global_step/sec: 245.909
INFO:tensorflow:loss = 0.04464214, step = 8501 (0.403 sec)
INFO:tensorflow:global_step/sec: 249.281
INFO:tensorflow:loss = 0.026176054, step = 8601 (0.401 sec)
INFO:tensorflow:global_step/sec: 251.231
INFO:tensorflow:loss = 0.006545673, step = 8701 (0.398 sec)
INFO:tensorflow:global_step/sec: 253.859
INFO:tensorflow:loss = 0.004882335, step = 8801 (0.395 sec)
INFO:tensorflow:global_step/sec: 252.219
INFO:tensorflow:loss = 0.0017491174, step = 8901 (0.398 sec)
INFO:tensorflow:global_step/sec: 245.795
INFO:tensorflow:loss = 0.009426334, step = 9001 (0.404 sec)
INFO:tensorflow:global_step/sec: 242.911
INFO:tensorflow:loss = 0.006284533, step = 9101 (0.414 sec)
INFO:tensorflow:global_step/sec: 240.947
INFO:tensorflow:loss = 0.025789212, step = 9201 (0.413 sec)
INFO:tensorflow:global_step/sec: 250.578
INFO:tensorflow:loss = 0.008820767, step = 9301 (0.399 sec)
INFO:tensorflow:global_step/se

INFO:tensorflow:loss = 0.007008184, step = 11501 (0.335 sec)
INFO:tensorflow:global_step/sec: 295.665
INFO:tensorflow:loss = 0.010162791, step = 11601 (0.338 sec)
INFO:tensorflow:global_step/sec: 293.313
INFO:tensorflow:loss = 0.018813595, step = 11701 (0.342 sec)
INFO:tensorflow:global_step/sec: 291.56
INFO:tensorflow:loss = 0.015704727, step = 11801 (0.345 sec)
INFO:tensorflow:global_step/sec: 287.693
INFO:tensorflow:loss = 0.0024252692, step = 11901 (0.347 sec)
INFO:tensorflow:global_step/sec: 290.885
INFO:tensorflow:loss = 0.0009222906, step = 12001 (0.344 sec)
INFO:tensorflow:global_step/sec: 281.944
INFO:tensorflow:loss = 0.012167728, step = 12101 (0.353 sec)
INFO:tensorflow:global_step/sec: 275.483
INFO:tensorflow:loss = 0.005968025, step = 12201 (0.366 sec)
INFO:tensorflow:global_step/sec: 251.376
INFO:tensorflow:loss = 0.005323849, step = 12301 (0.394 sec)
INFO:tensorflow:global_step/sec: 246.5
INFO:tensorflow:loss = 0.00064170285, step = 12401 (0.405 sec)
INFO:tensorflow:glob

...

INFO:tensorflow:global_step/sec: 245.305
INFO:tensorflow:loss = 0.0017592767, step = 29901 (0.410 sec)
INFO:tensorflow:global_step/sec: 238.743
INFO:tensorflow:loss = 0.0018576537, step = 30001 (0.417 sec)
INFO:tensorflow:global_step/sec: 246.102
INFO:tensorflow:loss = 0.00014682826, step = 30101 (0.409 sec)
INFO:tensorflow:global_step/sec: 246.74
INFO:tensorflow:loss = 0.00084901735, step = 30201 (0.406 sec)
INFO:tensorflow:global_step/sec: 238.39
INFO:tensorflow:loss = 8.4567706e-05, step = 30301 (0.415 sec)
INFO:tensorflow:global_step/sec: 244.573
INFO:tensorflow:loss = 0.00063006236, step = 30401 (0.410 sec)
INFO:tensorflow:global_step/sec: 246.921
INFO:tensorflow:loss = 0.00038873433, step = 30501 (0.404 sec)
INFO:tensorflow:global_step/sec: 245.966
INFO:tensorflow:loss = 0.00046472717, step = 30601 (0.408 sec)
INFO:tensorflow:global_step/sec: 246.549
INFO:tensorflow:loss = 0.0012423642, step = 30701 (0.408 sec)
INFO:tensorflow:global_step/sec: 239.213
INFO:tensorflow:loss = 0.000

INFO:tensorflow:loss = 0.0008797346, step = 32901 (0.366 sec)
INFO:tensorflow:global_step/sec: 278.324
INFO:tensorflow:loss = 0.00020593316, step = 33001 (0.357 sec)
INFO:tensorflow:global_step/sec: 279.406
INFO:tensorflow:loss = 0.005040213, step = 33101 (0.360 sec)
INFO:tensorflow:global_step/sec: 282.534
INFO:tensorflow:loss = 0.00123941, step = 33201 (0.352 sec)
INFO:tensorflow:global_step/sec: 276.557
INFO:tensorflow:loss = 0.000105511375, step = 33301 (0.361 sec)
INFO:tensorflow:global_step/sec: 281.974
INFO:tensorflow:loss = 0.0005436877, step = 33401 (0.357 sec)
INFO:tensorflow:global_step/sec: 275.185
INFO:tensorflow:loss = 0.00048338843, step = 33501 (0.360 sec)
INFO:tensorflow:global_step/sec: 276.942
INFO:tensorflow:loss = 0.0002278577, step = 33601 (0.365 sec)
INFO:tensorflow:global_step/sec: 261.299
INFO:tensorflow:loss = 0.0003909147, step = 33701 (0.382 sec)
INFO:tensorflow:global_step/sec: 252.366
INFO:tensorflow:loss = 0.00015306745, step = 33801 (0.394 sec)
INFO:tens

INFO:tensorflow:global_step/sec: 244.079
INFO:tensorflow:loss = 0.0004105543, step = 36001 (0.414 sec)
INFO:tensorflow:global_step/sec: 248.627
INFO:tensorflow:loss = 4.3648874e-05, step = 36101 (0.399 sec)
INFO:tensorflow:global_step/sec: 247.02
INFO:tensorflow:loss = 0.0011041836, step = 36201 (0.406 sec)
INFO:tensorflow:global_step/sec: 268.146
INFO:tensorflow:loss = 0.0005190992, step = 36301 (0.370 sec)
INFO:tensorflow:global_step/sec: 260.004
INFO:tensorflow:loss = 8.7097695e-05, step = 36401 (0.385 sec)
INFO:tensorflow:global_step/sec: 246.054
INFO:tensorflow:loss = 0.00069148926, step = 36501 (0.407 sec)
INFO:tensorflow:global_step/sec: 249.971
INFO:tensorflow:loss = 0.0003187726, step = 36601 (0.402 sec)
INFO:tensorflow:global_step/sec: 249.19
INFO:tensorflow:loss = 0.0001492413, step = 36701 (0.400 sec)
INFO:tensorflow:global_step/sec: 242.63
INFO:tensorflow:loss = 0.00059616426, step = 36801 (0.412 sec)
INFO:tensorflow:global_step/sec: 244.497
INFO:tensorflow:loss = 0.000167

INFO:tensorflow:loss = 0.0006938551, step = 39001 (0.358 sec)
INFO:tensorflow:global_step/sec: 252.728
INFO:tensorflow:loss = 5.0548715e-05, step = 39101 (0.395 sec)
INFO:tensorflow:global_step/sec: 243.734
INFO:tensorflow:loss = 0.00063275173, step = 39201 (0.407 sec)
INFO:tensorflow:global_step/sec: 245.212
INFO:tensorflow:loss = 0.00046886335, step = 39301 (0.408 sec)
INFO:tensorflow:global_step/sec: 245.355
INFO:tensorflow:loss = 0.00069088506, step = 39401 (0.411 sec)
INFO:tensorflow:global_step/sec: 229.436
INFO:tensorflow:loss = 0.00015189774, step = 39501 (0.437 sec)
INFO:tensorflow:global_step/sec: 248.598
INFO:tensorflow:loss = 0.00094625744, step = 39601 (0.398 sec)
INFO:tensorflow:global_step/sec: 245.234
INFO:tensorflow:loss = 0.000936589, step = 39701 (0.408 sec)
INFO:tensorflow:global_step/sec: 239.121
INFO:tensorflow:loss = 0.00081782864, step = 39801 (0.418 sec)
INFO:tensorflow:global_step/sec: 236.659
INFO:tensorflow:loss = 0.00049395865, step = 39901 (0.423 sec)
INFO

157.64876794815063

In [38]:
#training accuracy
dnn_clf.score(X_train_mscaled,y_train)

INFO:tensorflow:Starting evaluation at 2018-08-05-15:03:51
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpJfpoIn/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-05-15:03:53
INFO:tensorflow:Saving dict for global step 40000: accuracy = 1.0, global_step = 40000, loss = 0.00049229775


{'accuracy': 1.0, 'global_step': 40000, 'loss': 0.00049229775}

In [39]:
yt_pred = dnn_clf.predict(X_train_mscaled)
accuracy_score(y_train, yt_pred['classes'])

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpJfpoIn/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


1.0

In [40]:
#test accuracy
dnn_clf.score(X_test_mscaled,y_test)

INFO:tensorflow:Starting evaluation at 2018-08-05-15:03:56
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpJfpoIn/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-05-15:03:56
INFO:tensorflow:Saving dict for global step 40000: accuracy = 0.983, global_step = 40000, loss = 0.07065034


{'accuracy': 0.983, 'global_step': 40000, 'loss': 0.07065034}

In [41]:
y_pred = dnn_clf.predict(X_test_mscaled)
accuracy_score(y_test, y_pred['classes'])

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpJfpoIn/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


0.983

Convolution Nueral net attempt (code from here: https://github.com/tflearn/tflearn/blob/master/examples/images/convnet_mnist.py)

In [2]:
!pip install tflearn

    100% |████████████████████████████████| 102kB 2.6MB/s 
  Running setup.py bdist_wheel for tflearn ... - done
  Stored in directory: /content/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn


In [0]:
from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

In [4]:
# Data loading and preprocessing
import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data(one_hot=True)
X = X.reshape([-1, 28, 28, 1])
testX = testX.reshape([-1, 28, 28, 1])

Succesfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting mnist/train-images-idx3-ubyte.gz
Succesfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting mnist/train-labels-idx1-ubyte.gz
Succesfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist/t10k-images-idx3-ubyte.gz
Succesfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [5]:
# Building convolutional network
network = input_data(shape=[None, 28, 28, 1], name='input')
network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = fully_connected(network, 128, activation='tanh')
network = dropout(network, 0.8)
network = fully_connected(network, 256, activation='tanh')
network = dropout(network, 0.8)
network = fully_connected(network, 10, activation='softmax')
network = regression(network, optimizer='adam', learning_rate=0.01,
                     loss='categorical_crossentropy', name='target')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
# Training
start = time.time()
model = tflearn.DNN(network, tensorboard_verbose=0)
model.fit({'input': X}, {'target': Y}, n_epoch=20,
           validation_set=({'input': testX}, {'target': testY}),
           snapshot_step=100, show_metric=True, run_id='convnet_mnist')
end = time.time()
end-start

Training Step: 17199  | total loss: 0.36394 | time: 148.427s
| Adam | epoch: 020 | loss: 0.36394 - acc: 0.9188 -- iter: 54976/55000
Training Step: 17200  | total loss: 0.36701 | time: 153.309s
| Adam | epoch: 020 | loss: 0.36701 - acc: 0.9144 | val_loss: 0.16060 - val_acc: 0.9616 -- iter: 55000/55000
--


3174.746565103531

In [9]:
#training accuracy
model.evaluate(X,Y)

[0.9551818181384694]

In [10]:
#test accuracy
model.evaluate(testX,testY)

[0.9616]